In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
data_dir='../input/'
train_data= pd.read_csv(data_dir+'train.csv')
train_data.head()

# Exploration

In [ ]:
print (train_data.shape)
print (train_data['ID'].nunique())

The data is unique for each row, so we cannot apply aggregation features such as max, sum, average.

In [ ]:
def check_nulls_data_types(train_data):
    nulls_=dict(1.0*train_data.isnull().sum()/100.0)
    nulls_cols=[]
    dtype_cols=[]
    for key,values in nulls_.items():
        if values>0.0:
            nulls_cols.append(key)
    dtype_= dict(train_data.dtypes)
    for k,v in dtype_.items():
        if v=='O':
            dtype_cols.append(k)
    
    return nulls_cols,dtype_cols

def is_outlier(points, thresh=3.5):
    """
    Returns a boolean array with True if points are outliers and False 
    otherwise.
    Parameters:
    -----------
        points : An numobservations by numdimensions array of observations
        thresh : The modified z-score to use as a threshold. Observations with
            a modified z-score (based on the median absolute deviation) greater
            than this value will be classified as outliers.
    Returns:
    --------
        mask : A numobservations-length boolean array.
    References:
    ----------
        Boris Iglewicz and David Hoaglin (1993), "Volume 16: How to Detect and
        Handle Outliers", The ASQC Basic References in Quality Control:
        Statistical Techniques, Edward F. Mykytka, Ph.D., Editor. 
    """
    if len(points.shape) == 1:
        points = points[:,None]
    median = np.median(points, axis=0)
    diff = np.sum((points - median)**2, axis=-1)
    diff = np.sqrt(diff)
    med_abs_deviation = np.median(diff)

    modified_z_score = 0.6745 * diff / med_abs_deviation

    return modified_z_score > thresh


In [ ]:
o=is_outlier(train_data['target'].values)

In [ ]:
train_data['outlier']=o
train_data['outlier'].value_counts()

In [ ]:
outlier= train_data.pop('outlier')

In [ ]:
null_cols,dtype_= check_nulls_data_types(train_data)

In [ ]:
print ("The null columns:",null_cols)
print ("The columns having dtpe as Object:",dtype_)

Obviously the only object column would be "ID", although I was expecting any category columns to fall in.

In [ ]:
y= train_data.pop('target')

In [ ]:
X= train_data
X.head()

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

In [ ]:
print ("Training set",X_train.shape)
print ("Training set target",y_train.shape)
print ("Test set",X_test.shape)
print ("Test set target",y_test.shape)
